In [6]:
import json
import glob
import os
import datetime
import re
import numpy as np
import pandas as pd
from pathlib import Path
from slack_data_loader2 import SlackLoader2, normalize_links
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.snowball import EnglishStemmer, RussianStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

import eli5
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn import linear_model
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import mean_squared_log_error, roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.grid_search import GridSearchCV

from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score

from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

from operator import itemgetter

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### data
* "channel"
* "start_message_text"
* "thread_text" (__eou__ __eot__)
* "message_type" = ["top", "first_thread", "thread"]
* "attachment" = 1|0
* "message_number" = (1 - top,)
* "message_text" 
### _target_
emoji, count

In [7]:
BASE_DIR = Path("../input/export_Feb_8_2018")

channels = ['_jobs', 'career']

loader = SlackLoader2(BASE_DIR, exclude_channels=[])#, only_channels=channels)
print(len(loader.messages))
#print(len(loader.threads))
#df = pd.DataFrame(loader.threads).set_index(["channel_name", "start_ts"])

751861


In [8]:
df = pd.DataFrame(loader.messages)

In [9]:
df.columns

Index(['attachments', 'bot_id', 'bot_link', 'channel', 'channel_id',
       'channel_name', 'client_msg_id', 'comment', 'display_as_bot', 'dt',
       'edited', 'file', 'hidden', 'icons', 'inviter', 'is_intro',
       'is_multiteam', 'is_org_shared', 'is_shared', 'is_thread_broadcast',
       'item', 'item_type', 'members', 'mrkdwn', 'name', 'new_broadcast',
       'no_notifications', 'old_name', 'parent_user_id', 'permalink',
       'plain_text', 'purpose', 'reactions', 'reactions_', 'replies',
       'reply_count', 'room', 'root', 'slog_is_mpdm', 'slog_is_self_dm',
       'slog_is_shared', 'slog_is_slackbot_dm', 'slog_local_team_id',
       'subtype', 'text', 'text_', 'thread_ts', 'timestamp', 'topic', 'ts',
       'type', 'unfurl_links', 'unfurl_media', 'unread_count', 'upload',
       'upload_reply_to', 'user', 'username'],
      dtype='object')

In [13]:
def add_nums(df, text_col):
    items = ["words", "commas", "bangs", "quotas"]
    patterns = {"words":'\S+', 
                "commas":'\,',
                "dots":'\.',
                "bangs":'\!',
                "questions":'\!',
                "quotas":'\"', 
                "apost":'\`'
               }
    for name, pat in patterns.items():
        df['num_' + name] = df[text_col].str.count(pat)
    df['avg_word'] = df[text_col].str.len() / (1 + df["num_words"])
    return df
df = add_nums(df, "text_")

In [14]:
df.describe()

,reply_count,ts,unread_count,num_quotas,num_words,num_bangs,num_commas,num_apost,avg_word,num_questions,num_dots
count,28191.000000,7.518610e+05,27921.000000,751779.000000,751779.000000,751779.000000,751779.000000,751779.000000,751779.000000,751779.000000,751779.000000
mean,8.869462,1.488792e+09,8.955231,0.088346,15.025485,0.039011,1.049773,0.076384,6.256219,0.039011,0.997376
std,31.052946,1.776089e+07,31.190423,0.800703,28.929707,0.330264,2.536914,0.778057,4.128241,0.330264,2.900127
min,0.000000,1.426147e+09,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1.480428e+09,2.000000,0.000000,5.000000,0.000000,0.000000,0.000000,5.100000,0.000000,0.000000
50%,4.000000,1.489646e+09,4.000000,0.000000,8.000000,0.000000,0.000000,0.000000,5.833333,0.000000,0.000000
75%,9.000000,1.501659e+09,9.000000,0.000000,16.000000,0.000000,1.000000,0.000000,6.444444,0.000000,1.000000
max,4036.000000,1.518104e+09,4036.000000,212.000000,2730.000000,158.000000,199.000000,94.000000,1107.500000,158.000000,244.000000


In [4]:
df.describe

CPU times: user 624 ms, sys: 52 ms, total: 676 ms
Wall time: 686 ms


In [5]:
%%time

def get_ktop_reactions(df, k=20):
    df["reactions_"] = df["reactions_"].apply(lambda x : dict(x) )
    df2 = df["reactions_"].apply(pd.Series)
    df3 = pd.concat([df, df2], axis=1).drop('reactions_', axis=1)
    react_stat = df3.drop(["msg_counter"], axis=1).describe().transpose().sort_values(["count"], ascending=False)
    #df_[df_["count"] >= 100] #get by threshold
    cols_to_del = react_stat[k:].index.values
    ktop_cols = react_stat[:k].index.values
    was_rows = df3.shape[0]
    df3 = df3.drop(cols_to_del, axis=1).dropna(axis=0, subset=ktop_cols, how='all')
    print("{} rows droped.".format(was_rows-df3.shape[0]))
    return df3, ktop_cols

reacts, ktop_cols = get_ktop_reactions(df_readed)

13408 rows droped.
CPU times: user 13.3 s, sys: 524 ms, total: 13.9 s
Wall time: 13.9 s


In [6]:
reacts

end_ts  \
channel_name        start_ts                                                
_general            2015-07-23 19:10:16.000091                        NaT   
                    2015-07-23 19:22:55.000097                        NaT   
                    2015-08-26 14:06:00.000012                        NaT   
_random_flood       2015-09-09 14:29:57.000056                        NaT   
                    2015-09-16 09:13:30.000283                        NaT   
                    2015-09-16 10:36:45.000314                        NaT   
_meetings           2015-09-18 13:56:34.000333                        NaT   
                    2015-09-18 20:21:33.000397                        NaT   
                    2015-09-19 01:00:43.000400                        NaT   
                    2015-09-21 17:32:12.000442                        NaT   
_random_flood       2015-09-22 14:19:34.000542                        NaT   
                    2015-09-23 15:08:59.000672                        NaT   
                    2015-09-25 01:21:11.000741                        NaT   
_meetings           2015-09-25 19:52:08.000505                        NaT   
interesting_links   2015-09-27 22:40:38.000152                        NaT   
_random_flood       2015-09-29 14:29:15.000927                        NaT   
_meetings           2015-09-29 15:31:52.000629                        NaT   
interesting_links   2015-09-29 20:02:32.000345                        NaT   
_meetings           2015-09-30 13:57:42.000683                        NaT   
_random_flood       2015-09-30 21:36:51.001016                        NaT   
                    2015-09-30 22:58:10.001065                        NaT   
_meetings           2015-10-01 17:56:09.000725                        NaT   
_random_flood       2015-10-01 21:10:09.001123                        NaT   
_meetings           2015-10-02 13:05:22.000735                        NaT   
_random_flood       2015-10-02 14:05:59.001193                        NaT   
interesting_links   2015-10-03 10:57:19.000372                        NaT   
_random_flood       2015-10-04 20:34:19.000055                        NaT   
                    2015-10-05 13:26:07.000118                        NaT   
                    2015-10-05 14:11:29.000126                        NaT   
_general            2015-10-06 15:45:33.000018                        NaT   
...                                                                   ...   
proj_kaggle_camera  2018-02-08 12:53:34.000025                        NaT   
                    2018-02-08 13:00:43.000167                        NaT   
mlcourse_open       2018-02-08 13:03:45.000045 2018-02-08 13:18:08.000489   
conference          2018-02-08 13:10:23.000198                        NaT   
nlp                 2018-02-08 13:42:33.000099 2018-02-08 13:56:24.000366   
_random_flood       2018-02-08 14:14:06.000234 2018-02-08 14:34:33.000237   
                    2018-02-08 14:14:06.000234 2018-02-08 14:41:12.000391   
proj_kaggle_camera  2018-02-08 14:31:20.000283                        NaT   
_random_flood       2018-02-08 14:44:16.000145 2018-02-08 14:48:06.000218   
career              2018-02-08 15:14:49.000257                        NaT   
                    2018-02-08 15:14:49.000257 2018-02-08 15:16:35.000015   
                    2018-02-08 15:14:49.000257 2018-02-08 15:19:59.000146   
                    2018-02-08 15:14:49.000257 2018-02-08 15:20:15.000314   
                    2018-02-08 15:14:49.000257 2018-02-08 15:29:52.000270   
                    2018-02-08 15:14:49.000257 2018-02-08 15:30:06.000352   
interesting_links   2018-02-08 15:19:17.000168                        NaT   
                    2018-02-08 15:19:17.000168 2018-02-08 16:15:16.000426   
theory_and_practice 2018-02-08 15:25:08.000161 2018-02-08 15:34:05.000008   
proj_kaggle_camera  2018-02-08 15:29:53.000149                        NaT   
career              2018-02-08 15:35:14.000121 2018-02-08 15:40

In [7]:
X = reacts['text'].fillna("_na_").values
y = reacts[ktop_cols].fillna(0).values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [8]:
%%time

ru_stemmer = RussianStemmer()
en_stemmer = EnglishStemmer()
analyzer = CountVectorizer().build_analyzer()

def is_russian_word(w):
    #if not isinstance(w, unicode):
    #    w = w.decode('utf8')
    for c in w:
        if ord(c) >= ord(u'а') and ord(c) <= ord(u'я'):
            return True
    return False

def stemmed_words(doc):
    for w in analyzer(doc):
        yield ru_stemmer.stem(w) if is_russian_word(w) else en_stemmer.stem(w)
        

vectorizer = TfidfVectorizer(min_df=5, analyzer=stemmed_words).fit(X_train)        

CPU times: user 3min 47s, sys: 276 ms, total: 3min 48s
Wall time: 3min 48s


### Other vectorizer

In [26]:
%%time

# we need a custom pre-processor to extract correct field,
# but want to also use default scikit-learn preprocessing (e.g. lowercasing)
default_preprocessor = CountVectorizer().build_preprocessor()

import re, string
re_tok = re.compile('([{}“”¨«»®´·º½¾¿¡§£₤‘’])'.format(string.punctuation))
def tokenize(s): return re_tok.sub(r' \1 ', s).split()
  
vectorizer = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1).fit(X_train)


CPU times: user 25.2 s, sys: 172 ms, total: 25.4 s
Wall time: 25.4 s


In [35]:
%%time

vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 5),
    max_features=20000).fit(X_train)

CPU times: user 1min 44s, sys: 556 ms, total: 1min 44s
Wall time: 1min 44s


In [36]:
for _word, _count in sorted(vectorizer.vocabulary_.items(),key=itemgetter(1),reverse=True)[:20]:
    print(_word,end=' ')
    print(_count)

•  19999
•	 19998
• 19997
”  19996
” 19995
“ 19994
’ 19993
—  19992
— 19991
–  19990
– 19989
яя 19988
яются 19987
яютс 19986
яют 19985
яю 19984
ящи 19983
яще 19982
ящ 19981
яч 19980


In [ ]:
%%time

def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    param_grid = {'C': Cs, 'dual': [False], 'penalty':['l1','l2'], 'class_weight': [None, 'balanced']}
    #param_grid = {'C': Cs}
    model = LinearSVC(random_state=0)
    grid_search = GridSearchCV(model, param_grid, cv=nfolds, n_jobs=1, scoring='roc_auc', verbose=1)
    grid_search.fit(X, y)
    #grid_search.best_params_
    return grid_search
  

gs_result = svc_param_selection(X_train_vec, y_train, 3)

print(gs_result.best_params_, gs_result.best_score_)

In [37]:
%%time

X_train_vec = vectorizer.transform(X_train)


X_val_vec = vectorizer.transform(X_val)

CPU times: user 1min 52s, sys: 240 ms, total: 1min 53s
Wall time: 1min 53s


In [38]:
def pr(y_i, y, x):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
  
def get_mdl(X, y):
    #y = y.values
    r = np.log(pr(1,y, X) / pr(0,y, X))
    m = LogisticRegression(C=4, dual=True)
    x_nb = X.multiply(r)
    return m.fit(x_nb, y), r  

def get_rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(np.expm1(y_true), np.expm1(y_pred)))

def get_roc_auc(y_true, y_pred):
    return roc_auc_score(y_true, y_pred)
  
def ridge():
    
    return RidgeClassifier(
        solver='auto',
        fit_intercept=True,
        alpha=0.5,
        max_iter=100,
        normalize=False,
        tol=0.05)




def linear_svm():
    C = 0.01
    return LinearSVC(penalty='l1', dual=False, C=C, random_state=0)

def get_y(y_train, index = 1):
    y_ = y_train[:,0]#.reshape(-1,1)
    return y_ #np.where(y_ > 0, 1, 0)

#X_train_vec = X_train_vec#.toarray()
y_ = get_y(y_train)

cv = KFold(n_splits=10, shuffle=False, random_state=42)
models = []
print("start...")
for train_ids, valid_ids in cv.split(X_train_vec):
    #model = linear_svm()
    #model = linear_model.BayesianRidge()
    model = linear_model.Ridge(alpha = .5)
    #model = linear_model.Lasso(alpha = 0.1)
    #model = linear_model.SGDRegressor()
    #model = linear_model.ElasticNet()
    #model = GradientBoostingRegressor()
    #RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
    model.fit(X_train_vec[train_ids], y_[train_ids])
    y_pred_valid = model.predict(X_train_vec[valid_ids])
    rmsle = get_rmsle(y_pred_valid, y_[valid_ids])
    print('valid rmsle: {}'.format(rmsle))
    models.append(model)
    
print("Make prediction")



test_predicts = np.ones(y_val.shape[0])
for m in models:
    test_predicts *= m.predict(X_val_vec)#.toarray())

test_predicts **= (1. / len(models))

print(get_rmsle(test_predicts, get_y(y_val)))

start...
valid rmsle: 5.021307792206184
valid rmsle: 4.812680746296698
valid rmsle: 5.9769607979782755
valid rmsle: 4.864922419616282
valid rmsle: 5.351521412230716
valid rmsle: 4.751220851243395
valid rmsle: 4.526285582022721
valid rmsle: 4.776532257711831
valid rmsle: 5.7520709497541835
valid rmsle: 5.150663601629282
Make prediction


/home/algis/.local/lib/python3.5/site-packages/ipykernel/__main__.py:68: RuntimeWarning: invalid value encountered in power


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [39]:
def get_rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(np.expm1(y_true), np.expm1(y_pred)))

def get_ridge():
    return Ridge(
        solver='auto',
        fit_intercept=True,
        alpha=0.5,
        max_iter=100,
        normalize=False,
        tol=0.05)

cv = KFold(n_splits=10, shuffle=True, random_state=42)
for train_ids, valid_ids in cv.split(X_train_vec):
    model = get_ridge()#RandomForestRegressor()
    model.fit(X_train_vec[train_ids], y_[train_ids])
    y_pred_valid = model.predict(X_train_vec[valid_ids])
    rmsle = get_rmsle(y_pred_valid, y_[valid_ids])
    print('valid rmsle: {}'.format(rmsle))
    break


valid rmsle: 6.1079038536165875


In [40]:
eli5.show_weights(model, vec=vectorizer)

Weight?,Feature
+17.295,_
+16.475,>.
+14.082,•
+13.947,•
+13.250,•
+13.169,•
+12.593,л ра
+12.187,e
+11.878,ь и п
+11.568,еку


In [43]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from scipy.special import logit, expit
from tqdm import tqdm

class_names = ktop_cols

train_text = X_train
test_text = X_val
all_text = np.concatenate([train_text, test_text])

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=15000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 5),
    max_features=20000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])


#submission = pd.DataFrame.from_dict(predictions)
#submission.to_csv('submission.csv', index=False)

NameError: name 'test' is not defined

In [ ]:
losses = []
#predictions = {'id': test['id']}

#y = reacts[ktop_cols].fillna(0).values


def get_ridge():
    return Ridge(
        solver='auto',
        fit_intercept=True,
        alpha=0.5,
        max_iter=100,
        normalize=False,
        tol=0.05)

for index, class_name in tqdm(enumerate(class_names)):
    #train_target = np.where(y_train[:,index] > 0, 1,0)
    train_target = y_train[:,index]
    #classifier = get_ridge()#LogisticRegression(solver='sag')
    classifier = linear_model.Ridge(alpha = .5)
    cv_loss = np.mean(cross_val_score(classifier, train_features, train_target, cv=5, scoring='neg_mean_squared_error'))
    losses.append(cv_loss)
    print('CV score for class {} is {}'.format(class_name, cv_loss))

    #classifier.fit(train_features, train_target)
    #predictions[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(losses)))



0it [00:00, ?it/s]
